# Optimal Portfolio Function

- Here I will create a function that creates an optimal portfolio using Non-negative least squares regression.
- This is how *Harvard* optimized their portfolio using target allocations based on historical returns and covariance between asset classes. This type of optimization is essential to long-term capital management because all investors care about is maximizing return while limiting risk. 
- This allocation isn't **the most efficient**, however, it is *the most realistic portfolio* because:
    1. It does not allow for short positions
    2. There is no leverage
    3. Portfolio Turnover is small
- The allocation weights you find are the suggested optimal weights to stick by in the long-run- obviously there will be times when the allocation goes above or below that *optimal* weight, but as long as you manage it, your returns will generally always be better than the unmanaged portfolio. 

- This means that these **optimal weights** indicate that you must rebalance the portfolio from time to time: selling what has grown to big and buying what has grown too small in your portfolio. 




In [3]:
# You are going to need to download some useful python packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm 
import matplotlib as mpl
import seaborn as sns
import scipy as scs
import math
import yfinance as yf
from statsmodels.regression.rolling import RollingOLS
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from statsmodels.regression.quantile_regression import QuantReg

from sklearn.linear_model import QuantileRegressor
from sklearn.decomposition import PCA

from scipy.optimize import lsq_linear
import warnings
warnings.filterwarnings("ignore")
plt.style.use("seaborn")
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline


In [4]:
# Here is a Helper Function for disecting univariate risk
def performance_summary(return_data, annualization = 12):
    """ 
        Returns the Performance Stats for given set of returns
        Inputs: 
            return_data - DataFrame with Date index and Monthly Returns for different assets/strategies.
        Output:
            summary_stats - DataFrame with annualized mean return, vol, sharpe ratio. Skewness, Excess Kurtosis, Var (0.5) and
                            CVaR (0.5) and drawdown based on monthly returns. 
    """
    summary_stats = return_data.mean().to_frame('Mean').apply(lambda x: x*annualization)
    summary_stats['Volatility'] = return_data.std().apply(lambda x: x*np.sqrt(annualization))
    summary_stats['Sharpe Ratio'] = summary_stats['Mean']/summary_stats['Volatility']

    summary_stats['Skewness'] = return_data.skew()
    summary_stats['Excess Kurtosis'] = return_data.kurtosis()
    summary_stats['VaR (0.05)'] = return_data.quantile(.05, axis = 0)
    summary_stats['CVaR (0.05)'] = return_data[return_data <= return_data.quantile(.05, axis = 0)].mean()
    
    wealth_index = 1000*(1+return_data).cumprod()
    previous_peaks = wealth_index.cummax()
    drawdowns = (wealth_index - previous_peaks)/previous_peaks

    summary_stats['Max Drawdown'] = drawdowns.min()
    summary_stats['Peak'] = [previous_peaks[col][:drawdowns[col].idxmin()].idxmax() for col in previous_peaks.columns]
    summary_stats['Bottom'] = drawdowns.idxmin()
    
    recovery_date = []
    for col in wealth_index.columns:
        prev_max = previous_peaks[col][:drawdowns[col].idxmin()].max()
        recovery_wealth = pd.DataFrame([wealth_index[col][drawdowns[col].idxmin():]]).T
        recovery_date.append(recovery_wealth[recovery_wealth[col] >= prev_max].index.min())
    summary_stats['Recovery'] = recovery_date
    
    return summary_stats

In [11]:
# I am going to use yahoo finance to download the data
# You need to always download this asset (the risk free rate) which has ticker "^IRX"-3 month treasuries which I will rescale to be monthly
# You need risk-free asset to compute the excess returns
tickers = 'AAPL COST HD SPY'
rf_ticker = '^IRX'

def NNLS_weights(tickers_assets = None, rf_asset = None, start_date = None, time_interval = '1mo',annual_factor = 12):
    """ 
        Returns the Optimal Allocation using Non-Negative Least Squares
        Inputs: 
            ticker_assets-is the variable ticker above
            rf_asset - is the variable rf_ticker above
            start_date - any date (e.g '2001-01-01')
            time_inverval - Will always be set to '1mo'
            annual_factor - 12 so that I can annualize the monthly data
        Output:
            port_weight - dataframe showing the optimal allocation of weights
            Annual Return - Annualized Return
            Risk - Annualized Volatility
            Sharpe Ratio - Sharpe Ratio of Individual Securities
            Skew - Skew of Securities (Should be 0 for Normally Distributed Variables)
            Kurtosis - Excess Kurtosis (Excess Kurtosis should also be 0 for Noramally Distributed Variables)
            VAR - 5% Monthly Historical Value at Risk
            CVAR - Average Monthly Value at Risk if Returns go below 5% historical VAR

    """
    asset_returns = pd.DataFrame(yf.download(tickers_assets, start = start_date, interval= time_interval)['Adj Close'].pct_change().dropna())
    rf_asset = pd.DataFrame(yf.download(rf_asset, start = start_date, interval= time_interval)['Adj Close'].dropna())
    rf_asset.columns = ['1M Treasury Bill']
    rf_asset = rf_asset*(1/100)
    rf_asset = rf_asset*(1/3)
    asset_returns = asset_returns.join(rf_asset, how = 'inner')
    excess_returns = asset_returns.subtract(asset_returns['1M Treasury Bill'], axis = 0).drop(columns = ['1M Treasury Bill'])
    mu_tilde = pd.DataFrame(excess_returns.mean()*annual_factor, columns = ['Annual Return'], dtype = 'float')
    mu_sigma = (excess_returns.std()*np.sqrt(annual_factor)).to_frame('Sigma')
    
    
    Ntime, Nassets = excess_returns.shape

    Y = np.ones((Ntime, 1))
    x = excess_returns
    beta = LinearRegression(fit_intercept=False, positive=True).fit(x,Y).coef_.transpose()
    beta /= beta.sum()
    beta = pd.DataFrame(beta, columns = ['Weight %'], index = [excess_returns.columns])
    beta = beta.set_index(beta.index)
    beta['Annual Return'] = mu_tilde['Annual Return']
    beta['Risk (sigma)'] = mu_sigma
    beta['Sharpe Ratio'] = beta['Annual Return']/beta['Risk (sigma)']
    beta['skew'] = excess_returns.skew()
    beta['kurtosis'] = excess_returns.kurtosis()
    beta['Value-At-Risk(.05)'] = excess_returns.quantile(.05, axis = 0)
    beta['CVAR(.05)'] = excess_returns[excess_returns <= excess_returns.quantile(.05, axis = 0)].mean()
    return beta
  





In [16]:
NNLS_weights(tickers, rf_ticker, '2003-01-01', '1mo')

[*********************100%***********************]  4 of 4 completed
[*********************100%***********************]  1 of 1 completed


,Weight %,Annual Return,Risk (sigma),Sharpe Ratio,skew,kurtosis,Value-At-Risk(.05),CVAR(.05)
AAPL,0.574592,0.331241,0.329107,1.006483,-0.207777,1.110108,-0.118981,-0.174101
COST,0.181486,0.128592,0.198395,0.648158,-0.217904,0.268226,-0.093023,-0.121714
HD,0.243922,0.136180,0.226907,0.600159,0.023917,0.305423,-0.088796,-0.133102
SPY,0.000000,0.057405,0.151310,0.379384,-0.455384,1.235082,-0.075625,-0.102284


- Some interesting results:
    1. Generally speaking, the higher the sharpe ratio for an individual security, the higher the allocation. However, we can see that home depot gets a higher allocation even though its Sharpe is smaller than Costco. This could be possible for two reason: Home Depot is outperforming Costco on a yearly basis, and or Home Depot is less correlated with Apple resulting in a diversification benefit. 

In [ ]:
# Here I will calculate the hypothetical portfolio statistics

def port_performance(returns_excess)